# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.3/872.3 kB 12.5 MB/s eta 0:00:00


Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [2]:
from ultralytics import YOLO, solutions
import numpy as np
import matplotlib.pyplot as plt
import cv2


## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [3]:
import urllib. request




url = 'https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8n.pt'
filename= 'YOLO_Model.pt'
urllib.request.urlretrieve(url, filename)

model = YOLO(filename)

In [4]:
model.info()

YOLOv8n summary: 225 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs


(225, 3157200, 0, 8.8575488)

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [19]:
# video capture
cap = cv2.VideoCapture('YOLO_Video.mp4')

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [23]:
# extract w,h, fps


w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))



## Prepare Video Writer to Store the Output

[link text](https://)Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [26]:
# video writer

video_writer = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (w,h))

In [28]:
success, frame = cap.read()
# Regions (4) of intrestes rectangle
# top
top_roi = cv2.rectangle(frame, (0, 0), (w, 50), (0, 255, 0), 2)
# bottom
bottom_roi = cv2.rectangle(frame, (0, h - 50), (w, h), (0, 255, 0), 2)
# left
left_roi = cv2.rectangle(frame, (0, 0), (50, h), (0, 255, 0), 2)
# right
right_roi = cv2.rectangle(frame, (w - 50, 0), (w, h), (0, 255, 0), 2)


# count vehicles on all sides

left_vehicle_count = 0
Right_vehicle_count = 0
top_vehicle_count = 0
bottom_vehicle_count = 0

while cap.isOpened():
    success, frame = cap.read()

    if success:

        results = model(frame, conf=0.5)
        annot_frame = results[0].plot()

        # draw ROI

        # top
        top_roi = cv2.rectangle(annot_frame, (0, 0), (w, 50), (0, 255, 0), 2)
        # bottom
        bottom_roi = cv2.rectangle(annot_frame, (0, h - 50), (w, h), (0, 255, 0), 2)
        # left
        left_roi = cv2.rectangle(annot_frame, (0, 0), (50, h), (0, 255, 0), 2)
        # right
        right_roi = cv2.rectangle(annot_frame, (w - 50, 0), (w, h), (0, 255, 0), 2)


        cv2.putText(annot_frame, f"Top: {top_vehicle_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(annot_frame, f"Bottom: {bottom_vehicle_count}", (10, h - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(annot_frame, f"Left: {left_vehicle_count}", (10, h // 2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        cv2.putText(annot_frame, f"Right: {Right_vehicle_count}", (w - 150, h // 2), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)


        video_writer.write(annot_frame)

    else:
        break


cap.release()
video_writer.release()
cv2.destroyAllWindows()

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [18]:
#  top-left (x1,y1) and bottom-right (x2,y2)

results = model(frame, conf=0.5)
boxes = results[0].boxes.xyxy.cpu().numpy()
labels = results[0].boxes.cls.cpu().numpy()
for box, label in zip(boxes, labels):
            x1, y1, x2, y2 = box.astype(int)


            if label in [1, 2,3]:  # class ID: 1 car,2 bus, 3 person

                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2


                if center_y < 50:  # top
                    top_vehicle_count += 1
                elif center_y > h - 50:  # bottom
                    bottom_vehicle_count += 1
                if center_x < 50:  # left
                    left_vehicle_count += 1
                elif center_x > w - 50:  # right
                    Right_vehicle_count += 1

WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/bus.jpg: 640x480 3 persons, 1 bus, 294.6ms
image 2/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/zidane.jpg: 384x640 2 persons, 229.7ms
Speed: 6.5ms preprocess, 262.1ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)


## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [16]:
left_vehicle_count = 0
right_vehicle_count = 0
top_vehicle_count = 0
bottom_vehicle_count= 0

line_position = 100 # try 100 first
line_color = (0, 140, 0)


In [17]:
# get help from: https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html

from google.colab.patches import cv2_imshow

while True:
    ret, frame = cap.read()
    if not ret:
        break


    results = model(frame)

    # vehicles on all sides will be in list
    left_vehicles = []
    right_vehicles = []
    top_vehicles = []
    bottom_vehicles = []

    # the process
    for result in results:
        if result.boxes:
            for box in result.boxes:
                if box.conf > 0.5:  # try 0.5 first
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    class_ID = int(box.cls[0])
                    confidence = box.conf[0] # show conf score

                    # where the vehicle locate, center_x for left&right vehicles
                    center_x = (x1 + x2) // 2
                    #display the calculated center of each vehicle

                    if center_x < frame.shape[1] // 2:
                        left_vehicles.append((x1, y1, x2, y2, confidence, class_ID))
                    else:
                        right_vehicles.append((x1, y1, x2, y2, confidence, class_ID))

                    # center_y for top&bottom vehicles
                    center_y = (y1 + y2) // 2
                    #display the calculated center of each vehicle

                    if center_x < frame.shape[1] // 2:
                        bottom_vehicles.append((x1, y1, x2, y2, confidence, class_ID))
                    else:
                        top_vehicles.append((x1, y1, x2, y2, confidence, class_ID))

                    # video should display your name on the top
                    # get help from: https://answers.opencv.org/question/232636/show-video-and-texts-on-single-window/
                    #title = 'Rahaf Almutairi'
                    #cv2.putText(frame,title,(100,100),cv2.FONT_HERSHEY_SIMPLEX, 1.5, (b,g,r),1,cv2.LINE_AA)


                    # https://www.geeksforgeeks.org/python-opencv-write-text-on-video/
                    cv2.putText(frame,'Rahaf Almutairi', (50, 50),font, 1,  (0, 255, 255),  2, cv2.LINE_4)


                    # surround objects with boundin box and label
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                    label = f"ID: {class_ID}, Confidence: {confidence:.2f}"
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_PLAIN, 0.5, (255, 255, 255), 1)




    # how many vehicles due to line postioin
    for vehicle in left_vehicles:
        if vehicle[1] > line_position:
            left_vehicle_count += 1
    for vehicle in right_vehicles:
        if vehicle[1] > line_position:
            right_vehicle_count += 1

    for vehicle in top_vehicles:
        if vehicle[1] > line_position:
            top_vehicle_count += 1
    for vehicle in bottom_vehicles:
        if vehicle[1] > line_position:
            bottom_vehicle_count += 1


    # the line to start count
    cv2.line(frame, (0, line_position), (frame.shape[1], line_position), line_color, 2)

    # show counts of vehicls
    cv2.putText(frame, f"Left Vehicles: {left_vehicle_count}", (10, 30),
                cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)
    cv2.putText(frame, f"Right Vehicles: {right_vehicle_count}", (10, 60),
                cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)
    cv2.putText(frame, f"Top Vehicles: {top_vehicle_count}", (10, 90),
                cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)
    cv2.putText(frame, f"Bottom Vehicles: {bottom_vehicle_count}", (10, 120),
                cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)


    cv2_imshow(frame)

    # to break
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
video_writer.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

Submit both the notebook and the output video